In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle


In [3]:
def load_data_from_pickle(obj_name):
    '''
    load_data_from_pickle function loads and deserializes an object from a pickle file.

    Args:
        obj_name (str): The path to the pickle file containing the serialized object. 

    Returns:
        obj (pd.DataFrame): The deserialized object that was stored in the pickle file. 

    '''
    with open(obj_name, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [4]:
X_train = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_X_train.pkl")
y_train = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_y_train.pkl")

In [5]:
X_test = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_X_test.pkl")
y_test = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_y_test.pkl")

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(585664, 18) (585664,) (193589, 18) (193589,)


In [7]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
train_df = pd.concat([X, y], axis=1)

majority_class = train_df[train_df['SepsisLabel'] == 0]
minority_class = train_df[train_df['SepsisLabel'] == 1]

majority_class_subset = majority_class.sample(n=2*len(minority_class))
train_df = pd.concat([majority_class_subset, minority_class])

X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

X = train_df.drop('SepsisLabel', axis=1)
y = train_df['SepsisLabel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((9395, 18), (9395,), (37576, 18), (37576,))

In [9]:
import requests

def send_post_request(url, data, headers=None):
    if headers is None:
        headers = {
            'Content-Type': 'application/json'
        }
    
    response = requests.post(url, json=data, headers=headers)
    
    if response.status_code == 200:
        print("Request was successful")
    else:
        print(f"Request failed with status code {response.status_code}")
    
    return response




In [10]:
url = "http://localhost:8085/predict" 
data = {
"data": X_test.iloc[:10,:].values.tolist(),
}

response = send_post_request(url, data)
print("Response:", response.json())


Request was successful
Response: {'predictions': [0, 0, 0, 0, 1, 0, 1, 0, 0, 0]}


In [12]:
X_test.iloc[0,:].values.tolist()

[-0.6478944656508806,
 0.2477501984834743,
 0.38259609079288515,
 -0.49008610186547774,
 -1.0701773176762106,
 -0.22003090353055435,
 1.0513384369512033,
 -0.41574693837126003,
 -1.1438867187926012,
 0.09602446907973813,
 0.23009028035426746,
 0.6317965084518705,
 0.2612444880484649,
 67.42,
 -5.33,
 13.0,
 0.0,
 1.0]

In [55]:
def batch_predict(url, X_test, batch_size=500):
    predictions = []
    num_batches = (len(X_test) + batch_size - 1) // batch_size 

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(X_test))
        batch_data = X_test.iloc[start_idx:end_idx].values.tolist()
        
        data = {
            "data": batch_data
        }
        
        response = send_post_request(url, data)
        if response.status_code == 200:
            predictions.extend(response.json().get('predictions', []))
        else:
            print(f"Batch {i+1} failed with status code {response.status_code}")
    
    return predictions

In [56]:
url = "http://localhost:8085/predict"

predictions = batch_predict(url, X_test)
print("Predictions:", predictions)

Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Request was successful
Predictions: [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 